# Kak's Three-Stage Protocol Demo
This notebook contains a demonstration of Kak's Three-Stage Protocol for quantum encryption. The implementation is broken down into the steps defined in our final paper, and indicates that the proposed methodology can be successfully followed for algorithm implementation.

The cells in this notebook can be run in sequence to show the implementation from end to end.

In [33]:
# Step 0 - imports
import numpy as np
from numpy import array, inf
from numpy.random import randint, randn, seed
from math import pi
from qiskit import(QuantumCircuit, execute, Aer, IBMQ, assemble, transpile)
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.providers.ibmq.job import job_monitor
from commpy.channelcoding.convcode import Trellis, conv_encode, viterbi_decode
import hashlib, hmac
import secrets
from Crypto.Cipher import Salsa20

print("Imports Successful")

Imports Successful


### Step 1: Stub definition
Here, we establish the basic structure of the algorithm and create a primary method which can be executed to encrypt a single bit message

In [34]:
def kak_3_stage(message):
    prepared = alice_prepare_message(message)
    transmission_1 = alice_apply_rotation(prepared)
    transmission_2 = bob_apply_rotation(transmission_1)
    transmission_3 = alice_remove_rotation(transmission_2)
    decrypted_message = bob_remove_rotation(transmission_3)
    return decrypted_message

### Step 2: External value initialization
Here, we set configuration values and establish secret keys for Alice and Bob

In [38]:
debug = True      # Change to False to remove messages
backend = 'qasm_simulator'
alice_key = np.random.uniform(0, 2*pi)
bob_key = np.random.uniform(0, 2*pi)
if debug:
    print("Alice's key: %s" % alice_key)
    print("Bob's key: %s" % bob_key)

Alice's key: 5.083624432299371
Bob's key: 1.035277866422189


### Step 3: Simplest possible implementations
In this step, we define simple operations at each stage of the protocol to initialize the quantum circuit and measure the result, but we do not provide actual implementations. To verify that something occurs in each step, we simply add an identity gate and print a debug message showing the full circuit.

In [36]:
def alice_prepare_message(message):
    if debug: print('Preparing message: %s' % message)
    qc = QuantumCircuit(1,1)
    if debug: 
        print('Alice prepared circuit')
        print(qc)
    return qc

def alice_apply_rotation(qc):
    qc.i(0)
    if debug: 
        print('After Alice applies rotation')
        print(qc)
    return qc
    
def bob_apply_rotation(qc):
    qc.i(0)
    if debug: 
        print('After Bob applies rotation')
        print(qc)
    return qc
    
def alice_remove_rotation(qc):
    qc.i(0)
    if debug: 
        print('After Alice removes rotation')
        print(qc)
    return qc

def bob_remove_rotation(qc):
    qc.i(0)
    qc.measure(0,0)
    if debug: 
        print('After Bob removes rotation and measures')
        print(qc)
    # Execute in simulator
    qasm_sim = Aer.get_backend(backend)
    qobj = assemble(qc, shots=1, memory=True)
    result = qasm_sim.run(qobj).result()
    measured_bit = int(result.get_memory()[0])
    
    return measured_bit

print(kak_3_stage(0))
print(kak_3_stage(1))

Preparing message: 0
Alice prepared circuit
     
q_0: 
     
c: 1/
     
After Alice applies rotation
     ┌───┐
q_0: ┤ I ├
     └───┘
c: 1/═════
          
After Bob applies rotation
     ┌───┐┌───┐
q_0: ┤ I ├┤ I ├
     └───┘└───┘
c: 1/══════════
               
After Alice removes rotation
     ┌───┐┌───┐┌───┐
q_0: ┤ I ├┤ I ├┤ I ├
     └───┘└───┘└───┘
c: 1/═══════════════
                    
After Bob removes rotation and measures
     ┌───┐┌───┐┌───┐┌───┐┌─┐
q_0: ┤ I ├┤ I ├┤ I ├┤ I ├┤M├
     └───┘└───┘└───┘└───┘└╥┘
c: 1/═════════════════════╩═
                          0 
0
Preparing message: 1
Alice prepared circuit
     
q_0: 
     
c: 1/
     
After Alice applies rotation
     ┌───┐
q_0: ┤ I ├
     └───┘
c: 1/═════
          
After Bob applies rotation
     ┌───┐┌───┐
q_0: ┤ I ├┤ I ├
     └───┘└───┘
c: 1/══════════
               
After Alice removes rotation
     ┌───┐┌───┐┌───┐
q_0: ┤ I ├┤ I ├┤ I ├
     └───┘└───┘└───┘
c: 1/═══════════════
                    
After Bob remov

### Step 4: Implement classical processing
This step is a NOOP as no classical component is present in this circuit.

### Step 5: Implement real quantum portion of algorithm
Finally, we replace the identity gates from step 3 with the rotation operators that are used in the real circuit.

In [37]:
def alice_prepare_message(message):
    if debug: print('Preparing message: %s' % message)
    qc = QuantumCircuit(1,1)
    if message == 1: # Added in step 4
        qc.x(0)
    if debug: 
        print('Alice prepared circuit')
        print(qc)
    return qc

def alice_apply_rotation(qc):
    qc.rz(alice_key, 0)
    if debug: 
        print('After Alice applies rotation')
        print(qc)
    return qc
    
def bob_apply_rotation(qc):
    qc.rz(bob_key, 0)
    if debug: 
        print('After Bob applies rotation')
        print(qc)
    return qc
    
def alice_remove_rotation(qc):
    qc.rz(-alice_key, 0)
    if debug: 
        print('After Alice removes rotation')
        print(qc)
    return qc

def bob_remove_rotation(qc):
    qc.rz(-bob_key, 0)
    qc.measure(0,0)
    if debug: 
        print('After Bob removes rotation and measures')
        print(qc)
    # Execute in simulator
    qasm_sim = Aer.get_backend(backend)
    qobj = assemble(qc, shots=1, memory=True)
    result = qasm_sim.run(qobj).result()
    measured_bit = int(result.get_memory()[0])
    
    return measured_bit

Preparing message: 0
Alice prepared circuit
     
q_0: 
     
c: 1/
     
After Alice applies rotation
     ┌────────────┐
q_0: ┤ RZ(2.6512) ├
     └────────────┘
c: 1/══════════════
                   
After Bob applies rotation
     ┌────────────┐┌───────────┐
q_0: ┤ RZ(2.6512) ├┤ RZ(2.856) ├
     └────────────┘└───────────┘
c: 1/═══════════════════════════
                                
After Alice removes rotation
     ┌────────────┐┌───────────┐┌─────────────┐
q_0: ┤ RZ(2.6512) ├┤ RZ(2.856) ├┤ RZ(-2.6512) ├
     └────────────┘└───────────┘└─────────────┘
c: 1/══════════════════════════════════════════
                                               
After Bob removes rotation and measures
     ┌────────────┐┌───────────┐┌─────────────┐┌────────────┐┌─┐
q_0: ┤ RZ(2.6512) ├┤ RZ(2.856) ├┤ RZ(-2.6512) ├┤ RZ(-2.856) ├┤M├
     └────────────┘└───────────┘└─────────────┘└────────────┘└╥┘
c: 1/═════════════════════════════════════════════════════════╩═
                                    

### Testing and verification
To confirm that the algorithm functions correctly we test it with both possible inputs and verify that the expected result is produced. For a more complicated algorithm, this could use a standard software testing practice of edge cases plus a normal value or two.

In [44]:
prev_debug = debug
debug = False
print('Testing and verification of algorithm:')
print('Expected value 0. Match? %s' % (0 == kak_3_stage(0)))
print('Expected value 1. Match? %s' % (1 == kak_3_stage(1)))
debug = prev_debug

Testing and verification of algorithm:
Expected value 0. Match? True
Expected value 1. Match? True
